In [1]:
from os.path import expanduser

import numpy as np
import pandas as pd
import scipy

from tqdm.notebook import tqdm

import folium

import warnings
warnings.filterwarnings('ignore')

In [2]:
delhi_data = pd.read_csv(expanduser("~/kdd24_brick_kilns/data/delhi_kilns.csv"))
delhi_data.head(2)

,ImageName,Latitude,Longitude
0,kiln_1.png,28.399170,77.631222
1,kiln_2.png,28.411609,77.619730


In [3]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c

    return distance

In [33]:
# Create a map object with center to Delhi
m = folium.Map(location=[28.784800, 77.366721], zoom_start=13)

# add scatter of the kilns
for i in range(0, len(delhi_data)):
    folium.CircleMarker(
        location=[delhi_data.iloc[i]['Latitude'], delhi_data.iloc[i]['Longitude']],
        radius=2,
        color='color',
        fill=True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(m)

# closest points
dist = scipy.spatial.distance.cdist(delhi_data[['Latitude', 'Longitude']].values, delhi_data[['Latitude', 'Longitude']].values)
# fill dioagonal with inf
np.fill_diagonal(dist, np.inf)
print(dist.shape)
closest_idx = np.argmin(dist, axis=0)
    
# connect closent points with a line
violations = []
for i, idx in tqdm(enumerate(closest_idx)):
    # add a line
    lat1, lon1 = delhi_data.iloc[i][['Latitude', 'Longitude']]
    lat2, lon2 = delhi_data.iloc[idx][['Latitude', 'Longitude']]
    distance = haversine_distance(lat1, lon1, lat2, lon2)
    if distance < 1:
        violations.extend([(lat1, lon1), (lat2, lon2)])
        color = "red"
    else:
        color = "black"
    
    folium.PolyLine([delhi_data.iloc[i][['Latitude', 'Longitude']], delhi_data.iloc[idx][['Latitude', 'Longitude']]], color=color, weight=0.5, opacity=1).add_to(m)

# add legend to the map
m.add_child(folium.LatLngPopup())

(762, 762)


0it [00:00, ?it/s]

In [31]:
len(violations), len(set(violations)), len(set(violations))/len(dist), len(dist)

(1368, 684, 0.8976377952755905, 762)

In [50]:
# Create a map object with center to Delhi
m = folium.Map(location=[28.784800, 77.366721], zoom_start=9)

# closest points
dist = scipy.spatial.distance.cdist(delhi_data[['Latitude', 'Longitude']].values, delhi_data[['Latitude', 'Longitude']].values)
# fill dioagonal with inf
np.fill_diagonal(dist, np.inf)
print(dist.shape)
closest_idx = np.argmin(dist, axis=0)
    
# connect closent points with a line
violations = []
for i, idx in tqdm(enumerate(closest_idx)):
    # add a line
    lat1, lon1 = delhi_data.iloc[i][['Latitude', 'Longitude']]
    lat2, lon2 = delhi_data.iloc[idx][['Latitude', 'Longitude']]
    distance = haversine_distance(lat1, lon1, lat2, lon2)
    if distance < 1:
        violations.extend([(lat1, lon1), (lat2, lon2)])
        color = "red"
    else:
        color = "black"
    
    # folium.PolyLine([delhi_data.iloc[i][['Latitude', 'Longitude']], delhi_data.iloc[idx][['Latitude', 'Longitude']]], color=color, weight=0.5, opacity=1).add_to(m)
    
    # add scatter of the kilns
    folium.CircleMarker(
        location=[lat1, lon1],
        radius=0.01,
        color=color,
        # fill=True,
        # fill_color='purple',
        # fill_opacity=0.6
    ).add_to(m)
    folium.CircleMarker(
        location=[lat1, lon1],
        radius=0.01,
        color=color,
        # fill=True,
        # fill_color=color,
        # fill_opacity=1.0,
        # add edgecolor
        edgecolor="green",
    ).add_to(m)

# add legend to the map
m.add_child(folium.LatLngPopup())

(762, 762)


0it [00:00, ?it/s]